In [398]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [399]:
def unique_value(data, col):
  return np.unique(data[:, col])

In [400]:
def class_count(data): #data = ['y' 'n' 'y']
  return np.array(np.unique(data, return_counts=True))[1]

In [401]:
def partition(data, col, value, label):
  true_rows, false_rows = list(), list()
  labels_true, labels_false = list(), list()
  for row in range(len(data)):
    if isinstance(value, (int, float)) and (data[row, col] >= value): 
        true_rows.append(data[row, :])
        labels_true.append(label[row])
    elif data[row, col] == value:
      true_rows.append(data[row, :])
      labels_true.append(label[row])
    else:
      false_rows.append(data[row, :])
      labels_false.append(label[row])
  return np.asarray(true_rows), np.asarray(labels_true), np.asarray(false_rows), np.asarray(labels_false)

In [402]:
def gini(data):
  """
  data: ['yes', 'no', 'yes', 'no']
  colunm 
  """
  impurity = 1
  counts = class_count(data)

  # print(counts)

  for num in counts:
    
    # print("num: ", int(num))
    # print('lendata: ', len(data))

    impurity -= (int(num) / len(data)) * (int(num) / len(data))
  return impurity

In [403]:
def findMaxInfoGain(data, label):
  currentGini = gini(label)
  
  # print(currentGini)

  max_gain = -99
  max_col_gain = -99
  max_value_gain = -99
  for col in range(data.shape[1]):
    values = unique_value(data, col)
    n = len(data)
    for value in values:
      true_rows, labels_true, false_rows, labels_false = partition(data, col, value, label)
      if (len(true_rows) == 0 or len(false_rows) == 0):
        continue
      total_leaf_gini = len(true_rows) / n * gini(labels_true) + len(false_rows) / n * gini(labels_false)
      info_gain = currentGini - total_leaf_gini
      
      # print("Value: ", value)
      # print("info_gain: ", info_gain)

      if (info_gain > max_gain):
        max_gain = info_gain
        max_col_gain = col
        max_value_gain = value
  return max_gain, max_col_gain, max_value_gain

In [404]:
# data = pd.DataFrame()
# data['COLOR'] = pd.Series(data=['YELLOW', 'YELLOW']).astype('category')
# data['DIAM'] = pd.Series(data=[3, 3])
# data['LABEL'] = pd.Series(data=['APPLE', 'LEMON']).astype('category')
# # print(data)
# X = data.to_numpy()[:, :-1]
# Y = data.to_numpy()[:, -1]
# # print(X)
# # print(Y)
# max_gain, max_col_gain, max_value_gain = findMaxInfoGain(X, Y)
# print(max_value_gain)

In [516]:
class Node:
  def __init__(self, feature = None, col = None, left = None, right = None, train_data = None, label = None, value=None):
    self.feature = feature
    self.left = left
    self.right = right
    self.col = col  
    self.train_data = train_data
    self.label = label
    self.value = value

  def fit(self):
    max_gain, max_col_gain, max_value_gain = findMaxInfoGain(self.train_data, self.label)
    if (max_gain == -99):
      count = np.array(np.unique(self.label, return_counts=True))
      c = count.shape[1]
      maxNum = -99
      maxF = None
      for i in range(c):
        if (int(count[1][i]) > maxNum):
          maxNum = int(count[1][i])
          maxF = count[0][i]
      self.value = maxF
      return self
    else:
      true_rows, labels_true, false_rows, labels_false = partition(self.train_data, max_col_gain, max_value_gain, self.label)
      self.feature = max_value_gain
      self.col = max_col_gain
      self.train_data = None
      self.label = None
      self.value = None
      self.left = Node(train_data=false_rows, label=labels_false)
      self.left.fit()
      self.right = Node(train_data=true_rows, label=labels_true)
      self.right.fit()

  def is_leaf(self):
    return self.left == None and self.right == None

  def run(self, X):
    if (self.is_leaf()):
      return self.value
    else:
      val = X[self.col]
      if isinstance(val, (int, float)) and (val >= self.feature):
        return self.right.run(X)
      elif (val == self.feature):
        return self.right.run(X)
      else:
        return self.left.run(X)

In [517]:
class DecisionTreeClassifier:
  def __init__(self, maxdepth = None, root=None):
    self.maxdepth = maxdepth
    self.root = root

  def fit(self, X, y):
    X = X.to_numpy()
    y = y.to_numpy()
    self.root = Node(train_data=X, label=y)
    self.root.fit()
    return self.root

  def predict(self, X):
    X = X.to_numpy()
    rr = X.shape[0]
    ans = list()
    for id_row in range(rr):
      ans.append(self.root.run(X[id_row, :]))
    return ans

In [518]:
def printNode(node,i = 1):
  print("-"*i, node.feature, " ", node.value, node.col)
  i = i + 1
  if (node.left != None):
    printNode(node.left, i)
  if (node.right != None):
    printNode(node.right, i)

In [519]:
tree = DecisionTreeClassifier()
r = tree.fit(X, Y)
printNode(r)

- NO   None 1
-- 18   None 2
--- None   NO None
--- NO   None 0
---- None   YES None
---- 35   None 2
----- None   YES None
----- None   YES None
-- NO   None 0
--- 50   None 2
---- None   NO None
---- None   NO None
--- None   NO None


In [520]:
# data_test = pd.DataFrame()
# data_test['LOVES POPCORN'] = pd.Series(data=['NO', 'YES']).astype('category')
# data_test['LOVES SODA'] = pd.Series(data=['YES', 'NO']).astype('category')
# data_test['AGE'] = pd.Series(data=[3, 16])
# data_test
# result = tree.predict(data_test)
# print(result)

['NO', 'NO']


# Test data

In [ ]:
data = pd.DataFrame()
data['LOVES POPCORN'] = pd.Series(data=['YES', 'YES', 'NO', 'NO', 'YES', 'YES', 'NO']).astype('category')
data['LOVES SODA'] = pd.Series(data=['YES', 'NO', 'YES', 'YES', 'YES', 'NO', 'NO']).astype('category')
data['AGE'] = pd.Series(data=[7, 12, 18, 35, 38, 50, 83])
data['LOVES COOL AS ICE'] = pd.Series(data=['NO', 'NO', 'YES', 'YES', 'YES', 'NO', 'NO']).astype('category')
data

,LOVES POPCORN,LOVES SODA,AGE,LOVES COOL AS ICE
0,YES,YES,7,NO
1,YES,NO,12,NO
2,NO,YES,18,YES
3,NO,YES,35,YES
4,YES,YES,38,YES
5,YES,NO,50,NO
6,NO,NO,83,NO


In [ ]:
# data = pd.DataFrame()
# data['COLOR'] = pd.Series(data=['GREEN', 'YELLOW', 'RED', 'RED', 'YELLOW']).astype('category')
# data['DIAM'] = pd.Series(data=[3, 3, 1, 1, 3])
# data['LABEL'] = pd.Series(data=['APPLE', 'APPLE', 'GRAPE', 'GRAPE', 'LEMON']).astype('category')
# data

,COLOR,DIAM,LABEL
0,GREEN,3,APPLE
1,YELLOW,3,APPLE
2,RED,1,GRAPE
3,RED,1,GRAPE
4,YELLOW,3,LEMON


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   LOVES POPCORN      7 non-null      category
 1   LOVES SODA         7 non-null      category
 2   AGE                7 non-null      int64   
 3   LOVES COOL AS ICE  7 non-null      category
dtypes: category(3), int64(1)
memory usage: 493.0 bytes


In [ ]:
X = data.iloc[:, :-1]
Y = data.iloc[:, -1]
print(X)

  LOVES POPCORN LOVES SODA  AGE
0           YES        YES    7
1           YES         NO   12
2            NO        YES   18
3            NO        YES   35
4           YES        YES   38
5           YES         NO   50
6            NO         NO   83


In [ ]:
print(Y)

0     NO
1     NO
2    YES
3    YES
4    YES
5     NO
6     NO
Name: LOVES COOL AS ICE, dtype: category
Categories (2, object): ['NO', 'YES']


In [ ]:
print(LowestGiniColumn(X))

(2, 12)


In [ ]:
max_gain, max_col_gain, max_value_gain = findMaxInfoGain(X, Y)

Value:  GREEN
info_gain:  -0.030000000000000027
Value:  RED
info_gain:  0.08666666666666656
Value:  YELLOW
info_gain:  -0.01333333333333342
Value:  1
info_gain:  0.0
Value:  3
info_gain:  0.08666666666666656


In [ ]:
print(max_value_gain)

RED
